In [85]:
# Imports
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, MaxPool2D
from tensorflow.keras.optimizers import Adam
from sklearn.decomposition import PCA
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import TensorBoard
import os
from datetime import datetime

In [3]:
# Load Data

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
n_train = X_train.shape[0]
n_test = X_test.shape[0]

# Normalise images to be in the range [-1, 1]
X_train = X_train / 127.5 - 1
X_test = X_test / 127.5 - 1

# Convert each 28x28 image into a 784 dimensional vector
features_count = np.prod(X_train.shape[1:])
X_train_flatened = X_train.reshape(n_train, features_count)
X_test_flatened = X_test.reshape(n_test, features_count)


#### --- Task 1 --- ####

In [ ]:
# PCA and Centroids 
# The centroid is calculated by averaging the coordinates of all the points in a cluster. This average gives you a single point that best represents the center of that cluster. 

# Reduce the dimensionality of the data to 2 dimensions
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_flatened)

# Create a scatter plot of the PCA data, colored by digit
fig = px.scatter(X_train_pca, x=0, y=1, color=y_train, title='PCA plot of the MNIST Dataset', width=1000, height=600)
fig.update_layout(xaxis_title='Principal Component 1', yaxis_title='Principal Component 2')

# Create a DataFrame with the PCA data and digit labels
df_pca = pd.DataFrame(X_train_pca, columns=['PC1', 'PC2'])
df_pca['digit'] = y_train

# Compute centroids for each class by taking the mean of PC1 and PC2
centroids = df_pca.groupby('digit')[['PC1', 'PC2']].mean()

# Colour mapping 
color_sequence = px.colors.qualitative.Plotly
unique_digits = sorted(df_pca['digit'].unique())
color_map = {digit: color_sequence[i % len(color_sequence)] for i, digit in enumerate(unique_digits)}

# Add centroids as larger markers, each colored according to its class
for digit, row in centroids.iterrows():
    fig.add_trace(
        go.Scatter(
                    x=[row['PC1']],
                    y=[row['PC2']],
                    mode='markers',
                    marker=dict(color=color_map[digit], size=15, symbol='diamond'),
                    name=f'Centroid {digit}'
                )
            )

fig.show()

In [ ]:
# Scree Plot - Shows the percentage of variance explained by each principal component

pca_full = PCA(n_components=50)
pca_full.fit(X_train_flatened)
variance_ratios = pca_full.explained_variance_ratio_
components = np.arange(1, len(variance_ratios) + 1)

df = pd.DataFrame({'Principal Component': components, 'Explained Variance': variance_ratios * 100})
df['Cumulative Variance'] = df['Explained Variance'].cumsum()

fig = px.bar(df, x='Principal Component', y='Explained Variance', title='Scree Plot & Cumulative Variance', labels={'Explained Variance': 'Percentage of Variance Explained'}, width=1000, height=500)
fig.add_scatter(x=df['Principal Component'], y=df['Cumulative Variance'], mode='lines+markers', name='Cumulative Variance', line=dict(color='red'))
fig.show()

In [18]:
# Questions / Notes

# Why is PCA a good option to visualise data?
# PCA is a good option to visualise data because it reduces the dimensionality of the data to 2 dimensions, which makes it easier to plot and understand.

# Observations
# Clustering of Classes - 784D to 2D space and visually see the differences between the different classes, which is not possible in the original 784D space.
# Separation of Certain Classes - Some digits form more isolated clusters. Eg digit 1 has a tight cluster. - Relatively simple and unique shape compared to other digits.
# Overlap Among Other Classes - Digit classes, such as 3, 5, and 8, have clusters that overlap considerably. This suggests that their differences may not be well captured by a linear projection onto the first two principal components.

# Q - Which classes can be linearly separated?
# A - 1 and 0 can be linearly separated. Where 3, 5 and 8 cannot be linearly separated.

# PCA Notes
# When you have high dimensional data, there are many directions in which the data can vary. The first principal component is the direction along which the data varies the most.
# By projecting your data onto the space defined by the top few principal components (often just two for visualisation), you reduce the dimensionality while retaining most of the information (variance) in the original data. 

#### --- Task 2 --- ####

In [19]:
# Main Functions


def prepare_data(digit_1, digit_2):

    # Convert each 28x28 image into a 784 dimensional vector
    features_count = np.prod(X_train.shape[1:])
    X_train_flatened = X_train.reshape(n_train, features_count)
    X_test_flatened = X_test.reshape(n_test, features_count)

    # Filter out for digit_1 and digit_2 for binary classification
    cond = (y_train == digit_1) + (y_train == digit_2)
    binary_x_train = X_train_flatened[cond, :]
    binary_y_train = y_train[cond] * 1.0

    # Normalise training labels
    binary_y_train[binary_y_train == digit_1] = -1
    binary_y_train[binary_y_train == digit_2] = 1

    # Filter out for digit_1 and digit_2 for binary classification
    cond_test = (y_test == digit_1) + (y_test == digit_2)
    binary_x_test = X_test_flatened[cond_test, :]
    binary_y_test = y_test[cond_test] * 1.0

    # Normalise test labels
    binary_y_test[binary_y_test == digit_1] = -1
    binary_y_test[binary_y_test == digit_2] = 1

    return binary_x_train, binary_y_train, binary_x_test, binary_y_test



def predict(x, w, b):

        # Compute the linear combination for each sample    
        z = np.dot(x, w) + b         

        # If z >= 0, predict 1, otherwise predict -1
        prediction = np.where(z >= 0, 1, -1)  

        return prediction



def run_epoch_perceptron(binary_x_train, binary_y_train, binary_x_test, binary_y_test, num_epochs=100, learning_rate=0.01):

    def train_perceptron(x_train, y_train, num_epochs, learning_rate):

        # Get the number of samples and features
        n_samples, n_features = x_train.shape

        # Initialise weights and bias to zero
        w = np.zeros(n_features)  
        b = 0.0                   
        
        # Lists to store accuracy values
        train_accuracies = []
        epochs = []
        
        # Batch of stochastic gradient descent
        for epoch in range(num_epochs):
            for i in range(n_samples):

                # Check if the sample is misclassified
                if y_train[i] * (np.dot(x_train[i], w) + b) <= 0:

                    # Update weights and bias using the perceptron rule
                    w += learning_rate * y_train[i] * x_train[i]
                    b += learning_rate * y_train[i]

            # Evaluate training progress at each epoch
            predictions = predict(x_train, w, b)
            accuracy = np.mean(predictions == y_train)
            train_accuracies.append(accuracy)
            epochs.append(epoch + 1)

        # Plot accuracy vs epochs
        fig = px.line(x=epochs, y=train_accuracies, title='Training Accuracy vs Epochs', labels={'x': 'Epoch', 'y': 'Accuracy'}, width=1000, height=500)
        fig.show()
        
        return w, b


    # Train the perceptron using the binary training data
    w, b = train_perceptron(binary_x_train, binary_y_train, num_epochs, learning_rate)

    # Predict on the training data
    train_predictions = predict(binary_x_train, w, b)
    train_accuracy = np.mean(train_predictions == binary_y_train)
    print('Final Training Accuracy:', train_accuracy)

    # Predict on the test data
    test_predictions = predict(binary_x_test, w, b)
    test_accuracy = np.mean(test_predictions == binary_y_test)
    print('Test Accuracy:', test_accuracy)

    return test_accuracy



def run_optimisation_perceptron(binary_x_train, binary_y_train, binary_x_test, binary_y_test, max_iters=1000, learning_rate=0.01, tolerance=1e-3):

    def optimise_perceptron(x, y, max_iters, learning_rate, tolerance):

        # Initialise variables
        iter = 0
        error = np.inf
        error_list = []
        n,m = x.shape
        rng = np.random.default_rng()
        w = rng.random(m)
        b = rng.random()

        # While the iteration is less than the maximum number of iterations and the error is greater than the tolerance
        while (iter <= max_iters) & (error > tolerance):
            
            # Predict all samples
            predictions = predict(x, w, b)
            
            # Identify misclassified samples
            misclassified_indices = np.where(predictions != y)[0]
            
            # Compute current error (fraction of misclassified samples)
            error = len(misclassified_indices) / n
            error_list.append(error)
            
            # If no misclassifications, we can stop early
            if len(misclassified_indices) == 0:
                break

            # Update w, b for each misclassified sample
            for i in misclassified_indices:
                w += learning_rate * y[i] * x[i]
                b += learning_rate * y[i]

            iter += 1

        return w, b, error_list


    # Optimise on the training set
    w_opt, b_opt, error_list = optimise_perceptron(binary_x_train, binary_y_train, max_iters, learning_rate, tolerance)

    # Evaluate on training
    train_pred = predict(binary_x_train, w_opt, b_opt)
    train_accuracy = np.mean(train_pred == binary_y_train)
    print('Final Training Accuracy:', train_accuracy)

    # Evaluate on test
    test_pred = predict(binary_x_test, w_opt, b_opt)
    test_accuracy = np.mean(test_pred == binary_y_test)
    print('Test Accuracy:', test_accuracy)

    # Error Curve
    df_error = pd.DataFrame({'Iteration': list(range(1, len(error_list) + 1)), 'Misclassification Error': error_list})
    fig_error = px.line(df_error, x='Iteration', y='Misclassification Error', title='Perceptron Training Error', markers=True, width=1000, height=500)
    fig_error.show()

    # Visualise the learned weights as an image
    w_image = w_opt.reshape(28, 28)
    fig_weights = px.imshow(w_image, color_continuous_scale='RdBu', title='Learned Weight Image', width=1000, height=500)
    fig_weights.show()

    return test_accuracy



In [ ]:
# Run

digits = {'sample_1': (1, 0), 'sample_2': (8, 3), 'sample_3': (5, 3), 'sample_4': (8, 7), 'sample_5': (2, 9)}
results = {}

for run, (digit_1, digit_2) in enumerate(digits.values()):
    print(f'\n\nRun: {run + 1 }: -- Training for digits {digit_1} and {digit_2} --\n\n')
    print(' -- Epoch Perceptron Training --\n')
    binary_x_train, binary_y_train, binary_x_test, binary_y_test = prepare_data(digit_1, digit_2)
    epoch_test_accuracy = run_epoch_perceptron(binary_x_train, binary_y_train, binary_x_test, binary_y_test)
    print('\n -- Optimisation Perceptron Training --\n')
    optimisation_test_accuracy = run_optimisation_perceptron(binary_x_train, binary_y_train, binary_x_test, binary_y_test)  
    results[f'run_{run + 1}'] = {'digit_1': round(digit_1, 0), 'digit_2': round(digit_2, 0), 'epoch_test_accuracy': round(epoch_test_accuracy, 2), 'optimisation_test_accuracy': round(optimisation_test_accuracy, 2)}

df = pd.DataFrame(results)
display(df)

#### --- Task 3 --- ####

In [79]:
# Plot Training and Testing Accuracy Curves

def plot(history):

    train_acc = history.history['accuracy'][-1] * 100
    test_acc = history.history['val_accuracy'][-1] * 100
    print(f'Training accuracy: {train_acc:.2f}%')
    print(f'Test accuracy: {test_acc:.2f}%')

    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Test Accuracy')
    plt.title('Training vs. Testing Accuracy')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


def multi_plot(names, histories):
    
    plt.figure(figsize=(15,5))

    for i in range(len(histories)):
        plt.plot(histories[i].history['val_accuracy'])
        
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(names, loc='upper left')
    axes = plt.gca()
    axes.set_ylim([0.98,1])
    plt.show()


# Set up TensorBoard logging
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,  # Log histograms of weights/biases every epoch
    write_graph=True,  # Visualise the model graph
    write_images=True, # Save images of weights
    update_freq='epoch', # Log metrics at the end of each epoch
)

In [80]:
# Load Data

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train/255
y_train = np.eye(10)[y_train]
X_test = X_test/ 255
y_test = np.eye(10)[y_test]

In [ ]:
# Base MLP Model 

model = Sequential([
    Flatten(input_shape=(28, 28)),         # Flatten 28x28 images to 784
    Dense(1000, activation='relu'),        # First hidden layer
    Dense(1000, activation='relu'),        # Second hidden layer
    Dense(10, activation='softmax')        # Output layer
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=50,
    epochs=10,
    validation_data=(X_test, y_test)
)

plot(history)

In [ ]:
# Second MLP Model 

model = Sequential([
    Flatten(input_shape=(28, 28)),        
    Dense(500, activation='relu'),
    Dense(500, activation='relu'),
    Dense(500, activation='relu'),        
    Dense(500, activation='relu'),  
    Dense(500, activation='relu'),        
    Dense(10, activation='softmax')        
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    batch_size=50,
    epochs=10,
    validation_data=(X_test, y_test)
)

plot(history)

#### --- Task 4 --- ####

In [ ]:
# Load and prepare data

(x_train, y_train), (x_test, y_test) = mnist.load_data()
n_train = x_train.shape[0]
n_test = x_test.shape[0]
print(f'Training images: {n_train}, Test images: {n_test}')

# Reshape and normalise the data
x_train = x_train.reshape((n_train, 28, 28, 1)) / 255.0
x_test = x_test.reshape((n_test, 28, 28, 1)) / 255.0

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# Base CNN Model 

model = Sequential([
    Conv2D(32, kernel_size=(4, 4), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    Conv2D(128, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=50, epochs=10, validation_data=(x_test, y_test), callbacks=[tensorboard_callback])

plot(history)

In [ ]:
# Optimsed CNN Model 1

model = Sequential([
    Conv2D(8, kernel_size=(4, 4), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    Conv2D(32, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=50, epochs=10, validation_data=(x_test, y_test))

plot(history)

model.summary()

### Multimodel Runs

In [64]:
# Run multiple models 

def multi_model_run(models, names):

    nets = len(models)

    histories = [0] * nets

    for j in range(nets):

        histories[j] = models[j].fit(x_train, y_train, batch_size=50, epochs=10, validation_data=(x_test, y_test))

        print('CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}'.format(
            names[j], 10, 
            max(histories[j].history['accuracy']),  
            max(histories[j].history['val_accuracy'])  
        ))

    return histories    

In [ ]:
# What is the optimal CNN depth?

nets = 4
models = [0] * nets
names = ['CNN-1', 'CNN-2', 'CNN-3', 'CNN-4']
histories = [0] * nets  

for j in range(nets):

    models[j] = Sequential()
    models[j].add(Conv2D(16, kernel_size=(4,4), strides=(1,1), padding='same', activation='relu', input_shape=(28,28,1)))
        
    if j>0:
        models[j].add(Conv2D(32, kernel_size=(4,4), strides=(2,2), padding='same', activation='relu'))
        
    if j>1:
        models[j].add(Conv2D(64, kernel_size=(4,4), strides=(2,2), padding='same', activation='relu'))

    if j>2:
        models[j].add(Conv2D(128, kernel_size=(4,4), strides=(2,2), padding='same', activation='relu'))

    models[j].add(Flatten())
    models[j].add(Dense(10, activation='softmax'))

    models[j].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


histories = multi_model_run(models, names)

multi_plot(names, histories)

In [ ]:
# What is the optimal number of feature maps (width)?

nets = 6
models = [0] * nets
names = ['8 maps','16 maps','24 maps','32 maps','48 maps','64 maps']

for j in range(6):

    models[j] = Sequential([
        Conv2D(j*8+8, kernel_size=(4,4), strides=(1,1), activation='relu', input_shape=(28,28,1)),
        Conv2D(j*16+16, kernel_size=(4,4), strides=(2,2), activation='relu'),
        Conv2D(j*32+32, kernel_size=(4,4), strides=(2,2), activation='relu'),
        Flatten(),
        Dense(10, activation='softmax')
    ])

    models[j].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

histories = multi_model_run(models, names)

multi_plot(names, histories)

In [ ]:
# What is the optimal dense layer size? ( beyond scope of this assignment)

nets = 8
model = [0] * nets
names = ['0N', '32N', '64N', '128N', '256N', '512N', '1024N', '2048N']


for j in range(nets):

    model[j] = Sequential()
    model[j].add(Conv2D(16, kernel_size=(4,4), strides=(1,1), activation='relu', input_shape=(28,28,1)))
    model[j].add(Conv2D(32, kernel_size=(4,4), strides=(2,2), activation='relu'))
    model[j].add(Flatten())
    
    if j>0:
        model[j].add(Dense(2**(j+4), activation='relu'))

    model[j].add(Dense(10, activation='softmax'))
    
    model[j].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


histories = multi_model_run(models, names)

multi_plot(names, histories)